# Lab: Bring your own script Challenge

## Introduction
Your new colleague in the data science team (who isn't very familiar with SageMaker) has written a nice notebook to tackle an image classification problem with Keras: "skLearn-Local Notebook.ipynb".

It works OK with the simple IRIS data set they were working on before, but now they'd like to take advantage of some of the features of SageMaker to tackle bigger and harder challenges.

Can you help refactor the Local Notebook code, to show them how to use SageMaker effectively?

## Getting Started

First, check you can run the  **sklearn-Local Notebook.ipynb**  notebook through - reviewing what steps it takes.

This notebook sets out a structure you can use to migrate code into, and lists out some of the changes you'll need to make at a high level. You can either work directly in here, or duplicate this notebook so you still have an unchanged copy of the original.

Try to work through the sections first with an MVP goal in mind (fitting the model to data in S3 via a SageMaker Training Job, and deploying/using the model through a SageMaker Endpoint). The goal is to understand the big picture on how you can bring your own code to SageMaker and scale your training and deploy. You can always have more advance models and more complex training code. 

The excercise of bringing your own training code to SageMaker is what we call ***'Script Mode'***. 

## Sklearn script mode training and serving
Script mode is a training script format for a number of supported frameworks that lets you execute the training script in SageMaker with minimal modification (read more details in this blog [Script mode](https://aws.amazon.com/blogs/machine-learning/bring-your-own-model-with-amazon-sagemaker-script-mode/)). The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native SKlearn support sets up training-related environment variables and executes your training script. Script mode supports training with a Python script, a Python module, or a shell script. 

## Dependencies
Listing all our imports at the start helps to keep the requirements to run any script/file transparent up-front, and is specified by nearly every style guide including Python's official [PEP 8](https://peps.python.org/pep-0008/#imports)

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
# TODO: What else will you need?
# Have a look at the documentation: https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html
# to see which libraries need to be imported to use sagemaker and the Sklearn estimator estimator
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

#from sagemaker import get_execution_role

##  Prepare the Data
We download the Iris data from UCI Machine Learning repository directly from the web. this is the url where you can get the data similar to what we did in the "sklearn-local Notebook.ipynb"

In [ ]:
#TODO: download the data from internet
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data


In [ ]:
#read in the data with the headers

data = pd.read_csv('iris.data', 
                   names=['sepal length', 'sepal width', 
                          'petal length', 'petal width', 
                          'label'])

data.to_csv('data.csv')

In [ ]:
#split the data into train and test
train,test= np.split(data.sample(frac=1, random_state=22), [int(0.7 * len(data))])
train.head()

## Set up the environment: Execution Role, Session and S3 Bucket
Now that we have downloaded and reduced the data in the local directory, we will need to upload it to Amazon S3 to make it available for Amazon Sagemaker training.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region.

- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the get_execution_role method from sagemaker python SDK.

In [ ]:
# TODO: This is where you can setup execution role, session and S3 bucket.

#define samemaker role 
sagemaker_session = sagemaker.Session()
#define sagemaker session
bucket = sagemaker_session.default_bucket()
#define default bucket
role = sagemaker.get_execution_role()

## Upload Data to Amazon S3
Next is the part where you need to upload the images to Amazon S3 for Sagemaker training. You can refer to the previous example on how to do it using the aws s3 sync CLI command or using the boto3 SDK. The high-level command aws s3 sync command synchronizes the contents of the target bucket and source directory. It allows the use of options such as --delete that allows to remove objects from the target that are not present in the source and --exclude or --include options that filter files or objects to exclude or not exclude.

⏰ Note: Uploading to Amazon S3 typically takes about 2-3 minutes assuming a reduction_factor of 2

In [ ]:
#TODO: import aws boto3 library
import boto3

#TODO:convert the test and train to CSV 
train=train.to_csv('train.csv')
test=test.to_csv('test.csv')

#TODO:upload the data on to your sagemaker defulat S3 bucket in a folder called training and your data called 'data.csv'
s3_session = boto3.Session().resource('s3')
s3_session.Bucket(bucket).Object('data/train/train.csv').upload_file('train.csv')
s3_session.Bucket(bucket).Object('data/test/test.csv').upload_file('test.csv')

## Data Input ("Channels") Configuration
The draft code has 2 data sets: One for training, and one for test/validation. 

In SageMaker terminology, each input data set is a "channel" and we can name them however we like... Just make sure you're consistent about what you call each one!

For a simple input configuration, a channel spec might just be the S3 URI of the folder. For configuring more advanced options, there's the s3_input class in the SageMaker SDK.

In [ ]:
# TODO: Define your 2 data channels (train and test)
# The data can be found in: "s3://{bucket_name}/mnist/training" and "s3://{bucket_name}/mnist/testing"
# We can use either the s3_input (which gives us additional configuration options), or a plain string:

train_channel = sagemaker.inputs.TrainingInput(f"s3://{bucket}/data/train/train.csv")

test_channel = f"s3://{bucket}/data/test/test.csv"


## Algorithm ("Estimator") Configuration and Run
Instead of loading and fitting this data here in the notebook, we'll be creating a Sklearn Estimator through the SageMaker SDK, to run the code on a separate container that can be scaled as required.

The ["Using SKlearn with the SageMaker Python SDK"](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#using-scikit-learn-with-the-sagemaker-python-sdk) docs give a good overview of this process. You should run your estimator in script mode (which is easier to follow than the old default legacy mode) and as Python 3.

Use the **main.py*** file already prepared for you in your local directory as your entry point to port code into - which has already been created for you with some basic hints.


In [ ]:
#TODO:define your estimator using SKlearn framework

sklearn_estimator = SKLearn(entry_point='main.py',
                            instance_type='ml.m4.xlarge',
                            framework_version='0.20.0',
                            role=role)

# Calling `fit`
When you're ready to try your script in a SageMaker training job, you can call estimator.fit() as we did in previous exercises:To start a training job, we call `estimator.fit(training_data_uri)`.

When training is complete, the training job will upload the saved model to S3 for deployment.

In [ ]:
#TODO:call the fit function and pass on your data you  uploaded to S3 above for the training to start
sklearn_estimator.fit({'train':train_channel, 'test': test_channel})

## Deploy and Use Your Model (Real-Time Inference)
We are now ready to deploy our model to Sagemaker hosting services and make real time predictions

In [ ]:
#TODO:deploy the model to a real time endpoint
predictor = sklearn_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)

Let's now send some data to our model to predict- the data shouldbe sent in the accepted format (The data sent to the endpoint for this model should be 'text.csv' format) and the code below just does that. We also ensure to perform the same processing on our test, same as what we did on our training data.

In [ ]:
#TODO:now get some test data to test your model and process them similar to our training set
from sklearn.preprocessing import StandardScaler
test=pd.read_csv("test.csv")
test=test.iloc[:,0:4].values.tolist()

sc = StandardScaler()
test = sc.fit_transform(test)
#print(test)

In [ ]:
#TODO:the body that you send to your model enpoint should be text/csv format, get your data to the right format before sending it to you model endpoint for prediciton
request_body = ""
for row in test:
    request_body += ",".join([str(n) for n in row]) + "\n"

In [ ]:
#TODO:now envoke your endpoint and get predictions
client = boto3.client('sagemaker-runtime')

endpoint=predictor.endpoint_name


content_type = "text/csv"

response = client.invoke_endpoint(
    EndpointName=endpoint,
    ContentType=content_type,
    Body=request_body
    )
response['Body'].read()